In [1]:
# Todd McCullough
# 2020
import numpy as np
import pandas as pd
import cpl_main as cpl 
import random
import re

In [2]:
import pickle
filename = 'models/cpl_match_classifier.sav'

In [3]:
cpl_classifier_model = pickle.load(open(filename, 'rb'))

In [4]:
regressor = 'models/cpl_score_regressor-07-20-vr---34.sav'
cpl_score_model = pickle.load(open(regressor, 'rb'))

In [5]:
year = '2020'
team_ref = pd.read_csv('datasets/teams.csv')
team_ref = team_ref[team_ref['year'] == int(year)]

In [6]:
results = pd.read_csv(f'datasets/{year}/cpl-{year}-results.csv')
results_old = pd.read_csv(f'datasets/{year}/cpl-{year}-results_old.csv')
stats = pd.read_csv(f'datasets/{year}/cpl-{year}-stats.csv')
stats_old = pd.read_csv(f'datasets/{year}/cpl-{year}-stats_old.csv')
stats_seed = pd.read_csv(f'datasets/{year}/cpl-{year}-stats-seed.csv')
player_info = pd.read_csv(f'datasets/{year}/player-{year}-info.csv')
#results_brief = pd.read_csv(f'datasets/{year}/cpl-{year}-results_brief.csv')
team_stats = pd.read_csv(f'datasets/{year}/cpl-{year}-team_stats.csv')
schedule = pd.read_csv(f'datasets/{year}/cpl-{year}-schedule.csv')
rated_forwards = pd.read_csv(f'datasets/{year}/cpl-{year}-forwards.csv')
rated_midfielders = pd.read_csv(f'datasets/{year}/cpl-{year}-midfielders.csv')
rated_defenders = pd.read_csv(f'datasets/{year}/cpl-{year}-defenders.csv')
rated_keepers = pd.read_csv(f'datasets/{year}/cpl-{year}-keepers.csv')

In [7]:
### not needed anymore
def get_team_comparison(data,q1,q2):
    # getting games with q1 in both home or away
    db = data[(data['home'] == q1) | (data['away'] == q1)]
    db = db.reset_index()
    db.pop('index')
    # filering down more to get only the games against q2
    db = db.sort_values(by=['m','d'],ascending=False)
    db = db[(db['home'] == q2) | (db['away'] == q2)]
    #print(db)
    db = db.reset_index()
    db.pop('index')
    if db.empty == True:
        db = pd.DataFrame([(0,0,0,0,q1,'D',q2,'D','empty',q1)],columns=['d','m','hs','as','home','hr','away','ar','summary','team'])
    if db.shape[0] > 15:
        return db.head(15)
    else:
        return db

In [8]:
t = 0

In [9]:
# home side
q1 = schedule.iloc[t]['home']
# away side
q2 = schedule.iloc[t]['away']
print(q1,q2)

Forge FC Cavalry FC


In [10]:
results_brief = cpl.get_results_brief(results,team_ref)
results_brief_old = cpl.get_results_brief(results_old,team_ref)
results_brief = pd.concat([results_brief,results_brief_old])
compare = cpl.get_team_comparison(results_brief,q1,q2)

In [11]:
compare

,d,m,hs,as,home,hr,away,ar,summary,team
0,12,5,1,2,Forge FC,L,Cavalry FC,W,L H 1 - 2 CFC,Forge FC
1,22,6,0,1,Cavalry FC,L,Forge FC,W,W A 0 - 1 CFC,Forge FC
2,13,8,0,0,Forge FC,E,Cavalry FC,E,E H 0 - 0 CFC,Forge FC
3,25,8,1,0,Forge FC,W,Cavalry FC,L,W H 1 - 0 CFC,Forge FC
4,9,10,2,1,Cavalry FC,W,Forge FC,L,L A 2 - 1 CFC,Forge FC
5,16,10,1,0,Forge FC,W,Cavalry FC,L,W H 1 - 0 CFC,Forge FC
6,26,10,1,0,Forge FC,W,Cavalry FC,L,W H 1 - 0 CFC,Forge FC
7,2,11,0,1,Cavalry FC,L,Forge FC,W,W A 0 - 1 CFC,Forge FC


In [12]:
t1_x, t1_y = cpl.get_NB_data(compare,q1)
t2_x, t2_y = cpl.get_NB_data(compare,q2)

In [13]:
game_info = schedule[schedule['home'] == q1]
game_info = game_info[game_info['away'] == q2]
game = game_info.iloc[0]['game']
game_h = cpl.get_home_away_comparison(stats,game,q1)
game_a = cpl.get_home_away_comparison(stats,game,q2)

In [14]:
### FIX THE ERROR

# https://stackoverflow.com/questions/40659212/futurewarning-elementwise-comparison-failed-returning-scalar-but-in-the-futur
### 

In [44]:
home_roster = cpl.best_roster(q1,results,results_old,stats,stats_old,stats_seed,player_info,rated_forwards)
away_roster = cpl.best_roster(q2,results,results_old,stats,stats_old,stats_seed,player_info,rated_forwards)

home_roster = cpl.get_compare_roster(results,q1,stats,team_ref,rated_forwards,rated_midfielders,rated_defenders,rated_keepers,player_info)
away_roster = cpl.get_compare_roster(results,q2,stats,team_ref,rated_forwards,rated_midfielders,rated_defenders,rated_keepers,player_info)

In [16]:
q1_roster = cpl.get_overall_roster(home_roster,player_info)
q2_roster = cpl.get_overall_roster(away_roster,player_info)

In [17]:
test = cpl.get_roster_overall('Forge FC',stats,team_ref,rated_forwards,rated_midfielders,rated_defenders,rated_keepers,player_info)
test['position'] = test['position'].astype('object')
test[test['position'] == 'g']

/home/todd/anaconda3/lib/python3.7/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


,image,first,last,number,position,overall,flag,link,name
2,empty.jpg,Baj,Maan,31,g,0.12,Canada.png,https://en.wikipedia.org/wiki/Forge_FC,Baj Maan
21,Triston-Henry.1.jpg,Triston,Henry,1,g,0.71,Canada.png,https://en.wikipedia.org/wiki/Triston_Henry,Triston Henry


In [18]:
home_win, draw, away_win = cpl.get_nb_match_prediction(q1,q2,t1_x,t1_y,t2_x,t2_y)
print(home_win, draw, away_win)

0.42 0.34 0.25


In [19]:
q1_roster_r = cpl.add_regressor_features(q1_roster)
q1_roster_r

,1,2,3,4,5,6,7,8,9,10,...,n9,n10,n11,n12,n13,n14,f,m,d,g
0,0.71,0.71,0.74,0.9,0.77,0.5,0.52,0.45,0.68,0.42,...,3,4,4,4,4,2,4,5,4,1


In [20]:
q2_roster_r = cpl.add_regressor_features(q2_roster)
q2_roster_r

,1,2,3,4,5,6,7,8,9,10,...,n9,n10,n11,n12,n13,n14,f,m,d,g
0,0.79,0.92,0.75,0.37,0.33,0.67,0.62,0.67,0.35,0.73,...,3,4,4,4,4,2,4,4,5,1


In [21]:
home_score, away_score, home_win, away_win = cpl.get_final_score_prediction(cpl_score_model,q1_roster_r,q2_roster_r,home_win,away_win)
print('home score',home_score,'home prob',home_win)
print('away score', away_score,'away prob', away_win)

home score 3 home prob 0.42
away score 2 away prob 0.25


In [22]:
hp,ap = cpl.get_classification(home_win, away_win,draw)
print(home_win, hp, home_score, away_win, ap, away_score)

0.42 3 3 0.25 1 2


In [23]:
score = home_score
team_pred = hp
q1_roster_c = cpl.add_classifier_features(q1_roster,score,team_pred)
q1_roster_c

,1,2,3,4,5,6,7,8,9,10,...,n9,n10,n11,n12,n13,n14,f,m,d,g
0,0.71,0.71,0.74,0.9,0.77,0.5,0.52,0.45,0.68,0.42,...,3,4,4,4,4,2,4,5,4,1


In [24]:
score = away_score
team_pred = ap
q2_roster_c = cpl.add_classifier_features(q2_roster,score,team_pred)
q2_roster_c

,1,2,3,4,5,6,7,8,9,10,...,n9,n10,n11,n12,n13,n14,f,m,d,g
0,0.79,0.92,0.75,0.37,0.33,0.67,0.62,0.67,0.35,0.73,...,3,4,4,4,4,2,4,4,5,1


In [40]:
new_cols1 = []
for counter in range(0,82):
    new_cols1.append(str(counter))

match_list = q1_roster_c.loc[0].tolist()
match_list.extend(q2_roster_c.loc[0].tolist())

home_array = pd.DataFrame([match_list],columns=new_cols1)

In [41]:
home_array = pd.concat([df,dg],axis=1)
home_array

,0,1,2,3,4,5,6,7,8,9,...,72,73,74,75,76,77,78,79,80,81
0,0.71,0.71,0.74,0.9,0.77,0.5,0.52,0.45,0.68,0.42,...,3.0,4.0,4.0,4.0,4.0,2.0,4.0,4.0,5.0,1.0


In [27]:
#### loss prediction seems to be the greatest outcome
# mostly ignoring it and just dealing with the win and draw comparsion, because that will point to the loser

In [28]:
k=[]

In [31]:
def get_final_game_prediction(model,home_array,home_score,away_score):
    
    def random_draw(x,y):
        if x > y:
            end = x
        else:
            end = y
        x = random.choice(range(end))
        y = x
        return x,y
    # get the model prediction probability of W, L, D
    
    ###### need to be sure of the order of these probabilities
    
    def roster_pred(model,array):
        prediction = model.predict([array.values[0]]).flatten()
        if prediction == 0:
            outcome = 'L'
        elif prediction == 1:
            outcome = 'D'
        else:
            outcome = 'W'
        return outcome
    
    def roster_prob(model,array):
        probability = model.predict_proba([array.values[0]]).flatten()
        return probability
    
    def norm(x,y,z):
        norm = x + y + z
        x, y, z = round(x / norm,2), round(y / norm,2), round(z / norm,2)
        return x, y, z
    
    
    # get the prediction from the classification model
    prediction = roster_pred(model,home_array)
    probability = roster_prob(model,home_array)
    print(prediction,probability[0],probability[1],probability[2],home_score,away_score,'\n') # for debug leave it
    k.append([probability[0],probability[1],probability[2]])
    #l.append([prediction,probability[0],probability[1],probability[2],home_score,away_score])
    p_l, p_d, p_w = probability[0],probability[1],probability[2]#norm(q1_w, q1_l, q1_d)
    
    #FIX THE SCORE ####
    # adjust score depending on the outcome from the prediction
    if prediction == 'W' and (home_score == away_score):
        away_score = random.choice(range(away_score-1))
    if prediction == 'L' and (home_score == away_score) or prediction == 'L' and (home_score > away_score):
        home_score = random.choice(range(away_score-1))
    if prediction == 'D':
        home_score, away_score = random_draw(home_score, away_score)
    
    return p_w, p_l, p_d, home_score, away_score

In [32]:
home_win_new, away_win_new, draw_new, home_score, away_score = get_final_game_prediction(cpl_classifier_model,home_array,home_score,away_score)
print(f'{q1} home win prob {home_win_new} \n Score: {home_score} \n{q2} away win prob {away_win_new} \n Score: {away_score} \ndraw prob {draw_new}')

W 0.35 0.15 0.5 3 2 

Forge FC home win prob 0.5 
 Score: 3 
Cavalry FC away win prob 0.35 
 Score: 2 
draw prob 0.15


In [42]:
k, l = [], []
for t in range(28):
    # home side
    q1 = schedule.iloc[t]['home']
    # away side
    q2 = schedule.iloc[t]['away']
    print('\n',q1,q2)
    compare = compare[compare['hr'] != 'E']
    t1_x, t1_y = cpl.get_NB_data(compare,q1)
    t2_x, t2_y = cpl.get_NB_data(compare,q2)

    game_info = schedule[schedule['home'] == q1]
    game_info = game_info[game_info['away'] == q2]
    game = game_info.iloc[0]['game']
    game_h = cpl.get_home_away_comparison(stats,game,q1)
    game_a = cpl.get_home_away_comparison(stats,game,q2)

    home_roster = cpl.best_roster(q1,results,results_old,stats,stats_old,stats_seed,player_info,rated_forwards)
    away_roster = cpl.best_roster(q2,results,results_old,stats,stats_old,stats_seed,player_info,rated_forwards)

    q1_roster = cpl.get_overall_roster(home_roster,player_info)
    q2_roster = cpl.get_overall_roster(away_roster,player_info)
    
    home_win, draw, away_win = cpl.get_nb_match_prediction(q1,q2,t1_x,t1_y,t2_x,t2_y)
    q1_roster_r = cpl.add_regressor_features(q1_roster)
    q2_roster_r = cpl.add_regressor_features(q2_roster)
    home_score, away_score, home_win, away_win = cpl.get_final_score_prediction(cpl_score_model,q1_roster_r,q2_roster_r,home_win,away_win)
    hp,ap = cpl.get_classification(home_win, away_win,draw)
    score = home_score
    team_pred = hp
    q1_roster_c = cpl.add_classifier_features(q1_roster,score,team_pred)
    score = away_score
    team_pred = ap
    q2_roster_c = cpl.add_classifier_features(q2_roster,score,team_pred)
    
    new_cols1 = []
    for counter in range(0,82):
        new_cols1.append(str(counter))

    match_list = q1_roster_c.loc[0].tolist()
    match_list.extend(q2_roster_c.loc[0].tolist())

    home_array = pd.DataFrame([match_list],columns=new_cols1)
    
    home_win_new, away_win_new, draw_new, home_score, away_score = get_final_game_prediction(cpl_classifier_model,home_array,home_score,away_score)
    print(f'{q1} home win prob {home_win_new} \n Score: {home_score} \n{q2} away win prob {away_win_new} \n Score: {away_score} \ndraw prob {draw_new}')


 Forge FC Cavalry FC
W 0.35 0.15 0.5 3 2 

Forge FC home win prob 0.5 
 Score: 3 
Cavalry FC away win prob 0.35 
 Score: 2 
draw prob 0.15

 York9 FC Atletico Ottawa
L 0.45 0.2 0.35 1 3 

York9 FC home win prob 0.35 
 Score: 1 
Atletico Ottawa away win prob 0.45 
 Score: 3 
draw prob 0.2

 HFX Wanderers FC Pacific FC
L 0.45 0.35 0.2 2 2 

HFX Wanderers FC home win prob 0.2 
 Score: 0 
Pacific FC away win prob 0.45 
 Score: 2 
draw prob 0.35

 Valour FC Cavalry FC
D 0.3 0.45 0.25 2 3 

Valour FC home win prob 0.25 
 Score: 0 
Cavalry FC away win prob 0.3 
 Score: 0 
draw prob 0.45

 Forge FC FC Edmonton
L 0.45 0.15 0.4 2 3 

Forge FC home win prob 0.4 
 Score: 2 
FC Edmonton away win prob 0.45 
 Score: 3 
draw prob 0.15

 Pacific FC York9 FC
D 0.25 0.45 0.3 2 1 

Pacific FC home win prob 0.3 
 Score: 1 
York9 FC away win prob 0.25 
 Score: 1 
draw prob 0.45

 Atletico Ottawa Valour FC
L 0.4 0.2 0.4 3 2 

Atletico Ottawa home win prob 0.4 
 Score: 0 
Valour FC away win prob 0.4 
 Score:

In [43]:
outcomes = pd.DataFrame(k,columns=['loss','draw','win'])
outcomes.describe()

,loss,draw,win
count,28.000000,28.000000,28.000000
mean,0.383929,0.298214,0.317857
std,0.075833,0.084418,0.080754
min,0.250000,0.150000,0.200000
25%,0.350000,0.250000,0.250000
50%,0.400000,0.300000,0.300000
75%,0.412500,0.350000,0.400000
max,0.550000,0.500000,0.500000


### old function to get data

In [ ]:
# used data differently, took the MNB() and combined it with the new prediction
# new model uses the NB() prediction as part of it's prediction, making this no longer necessary

def get_final_game_prediction(model,q1_roster,q2_roster,home_win,away_win,draw):
    q1_prediction = roster_pred(model,q1_roster)
    q1_p = round(q1_prediction.iloc[2][0],2)
    q2_prediction = roster_pred(model,q2_roster)
    q2_p = round(q2_prediction.iloc[2][0],2)
    q_draw = (q1_prediction.iloc[1][0] + q2_prediction.iloc[2][0]) / 2
    total_ = q1_p + home_win + q2_p + away_win + q_draw + draw
    h_w = round((q1_p + home_win) / total_, 2)
    a_w = round((q2_p + away_win) / total_, 2)
    g_d = round((q_draw + draw) / total_, 2)
    return h_w, a_w, g_d

In [ ]:
home_win_new, away_win_new, draw_new = get_final_game_prediction(cpl_classifier_model,q1_roster_c,q2_roster_c,home_win,away_win,draw)
print(f'{q1} win probability: {round(home_win_new,2)} \n{q2} win probability: {round(away_win_new,2)} \nDraw probability: {round(draw_new,2)}')

### combining all the functions to clean it all up

This is a WIP and does not work currently

In [ ]:
def get_match_prediction(q1,q2,t1_x,t1_y,t2_x,t2_y,q1_roster,q2_roster,cpl_score_model,cpl_classifier_model):
    
    def get_match_prediction(q1,q2,x1,y1,x2,y2):

        def get_nb_prediction(query,x,y,result):
            mnb = MultinomialNB()
            # Train the model using the training sets
            mnb.fit(x,y)
            # use below instead of predicted = model.predict([result]) because we want the probability
            mnb_pred = np.round(mnb.predict_proba([result])[:, 1],decimals=2)
            pred = round(mnb_pred[0],2)
            #print(gnb_pred[0], bnb_pred[0], pred)
            return pred

        def get_match_prediction_result(query,x,y,array):
            prediction = get_nb_prediction(query,x,y,array)
            return prediction

        if (len(x1) == 0) or (len(x2) == 0):
            x = round(1/3,2)
            home_win, away_win,draw = x,x,x
            return home_win,away_win,draw
        home_win = get_match_prediction_result(q1,x1,y1,[1,2])
        draw = get_match_prediction_result(q1,x1,y1,[1,1])
        away_win = get_match_prediction_result(q2,x2,y2,[2,2])
        return home_win, draw, away_win
    
    def add_features(data,score,team_pred):
        cols = ['0','1','2','3','4','5','6','7','8','9','10','11','12','13']
        data = pd.DataFrame(data.values,columns=cols)
        data = data.astype('float32')
        data['score'] = score
        data['pred'] = team_pred
        data['sum'] = round(data[cols].sum(axis=1),2)
        data['count'] = (data[data[cols]>0].count(axis=1))-10
        data['result'] = round(data['sum'] / (data['count']+0.1),2)
        data['plus'] = data['sum'] + data['score']
        data['test'] = round(data['sum']  / data['sum'].max() - 0.05,3)
        if data['count'].values[0] == 0:
            data['test2'] = 0
        else:
            data['test2'] = round(data['count']  / data['count'].max() - 0.05,3)
        data['diff'] = round(data['score'] * data[cols].max(axis=1)).astype('int64') + 1
        return data

    def get_final_score_prediction(model,q1_roster,q2_roster,home_win_new,away_win_new):

        def roster_pred(model,array):
            prediction = model.predict([array]).flatten()
            return prediction

        def final_score_fix(home_score,away_score,home_win_new,away_win_new):
            if home_win_new > away_win_new and home_score < away_score: # fix the score prediction - if the probability of home win > away win and score doesn't reflect it
                print(home_score,away_score)
                old = home_score
                home_score = away_score # change the predicted score to reflect that
                away_score = old
                return home_score,away_score
            elif home_win_new < away_win_new and home_score > away_score: # else the probability of home win < away win
                print(home_score,away_score)
                old = away_score
                away_score = home_score
                home_score = old # change the predicted score to reflect that
                return home_score,away_score
            elif home_win_new < away_win_new and home_score == away_score:
                print(home_score,away_score)
                home_score = home_score - 1
                return home_score,away_score
            elif home_win_new > away_win_new and home_score == away_score:
                print(home_score,away_score)
                away_score = away_score - 1
                return home_score,away_score
            elif home_win_new == away_win_new:
                away_score = home_score
                print(home_score,away_score)
                return home_score,away_score
            else:
                print(home_score,away_score)
                return home_score,away_score

        def score(num): #improve this later for greater predictions
            new_score = int(round(num,0)) # convert the float value to int and round it
            return new_score

        q1_pred = roster_pred(model,q1_roster)
        q1_s = score(q1_pred[0])
        q2_pred = roster_pred(model,q2_roster)
        q2_s = score(q2_pred[0])
        home_score, away_score = final_score_fix(q1_s, q2_s,home_win_new,away_win_new)
        return home_score, away_score
    
    def get_classification(home_win, away_win,draw):
        if home_win > away_win:
            if home_win > draw:
                hp, ap = 3, 1
            else:
                hp, ap = 2, 2
        elif away_win > home_win:
            if away_win > draw:
                hp, ap = 3, 1
            else:
                hp, ap = 2, 2
        else:
            hp, ap = 2, 2
        return hp, ap

    def roster_pred(model,array):
        prediction = model.predict_proba([array.values[0]]).flatten()
        df = pd.DataFrame(prediction)
        return df

    def game_prediction(model,q1_roster,q2_roster,home_score,away_score):
        # get the prediction from the classification model
        q1_prediction = roster_pred(model,q1_roster)
        q2_prediction = roster_pred(model,q2_roster)
        # calculate the totals from win and draw predictions
        total = q1_prediction.iloc[2][0] + q1_prediction.iloc[1][0] + q2_prediction.iloc[2][0] + q2_prediction.iloc[1][0]
        # calculate the final probability using the win pred divided by the total
        q1_p = round(q1_prediction.iloc[2][0]/total,2)
        q2_p = round(q2_prediction.iloc[2][0]/total,2)
        q_draw = round(1 - q1_p - q2_p,2)

        # adjust score depending on the outcome from the prediction
        if (q1_p > q2_p) and (home_score == away_score):
            away_score = away_score - 1
        if q1_p > q2_p:
            if (q1_p - q2_p) > 0.5:
                away_score = away_score - 1
        if (q2_p > q1_p) and (home_score == away_score):
            home_score = home_score - 1
        if q2_p > q1_p:
            if (q2_p - q1_p) < 0.3:
                home_score = home_score - 1
        return q1_p, q2_p, q_draw, home_score, away_score
    
    home_win, draw, away_win = get_match_prediction(q1,q2,t1_x,t1_y,t2_x,t2_y)
    home_score, away_score = cpl.get_final_score_prediction(cpl_score_model,q1_roster,q2_roster,home_win,away_win)
    home_pred_result,away_pred_result = get_classification(home_win, away_win)
    q1_roster = add_features(q1_roster,home_score,home_pred_result)
    q2_roster = add_features(q2_roster,away_score,away_pred_result)
    home_win_new, away_win_new, draw_new, home_score, away_score = game_prediction(cpl_classifier_model,q1_roster,q2_roster,home_score,away_score)
    return home_win_new, away_win_new, draw_new, home_score, away_score

In [ ]:
home_win_new, away_win_new, draw_new, home_score, away_score = get_match_prediction(q1,q2,t1_x,t1_y,t2_x,t2_y,q1_roster,q2_roster,cpl_score_model,cpl_classifier_model)

print(f'{q1} home win prob {home_win_new} \n Score: {home_score} \n{q2} away win prob {away_win_new} \n Score: {away_score} \ndraw prob {draw_new}')